<a href="https://colab.research.google.com/github/mtpl85/PYPrinciples_Berkeley_Examples/blob/main/Assignment_2_SchoolAcceptance_Data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df1 =  pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", delimiter='\t')
df2 =  pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", delimiter='\t')
df3 =  pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", delimiter='\t')
#df3 = 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = df2['directors']
print(corpus[:2])
X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
# df2['directors_list'] = df2['direct']

0    nm0005690
1    nm0721526
Name: directors, dtype: object


In [ ]:
print(type(X))
print(X[:20])
# np.concat(X.toarray())

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 2206)	1
  (1, 69305)	1
  (2, 69305)	1
  (3, 69305)	1
  (4, 2206)	1
  (5, 2206)	1
  (6, 2206)	1
  (6, 37137)	1
  (7, 2206)	1
  (8, 10213)	1
  (9, 51527)	1
  (10, 77161)	1
  (11, 51527)	1
  (11, 51526)	1
  (12, 51527)	1
  (13, 51527)	1
  (13, 34902)	1
  (14, 69305)	1
  (15, 51527)	1
  (16, 77161)	1
  (16, 217123)	1
  (17, 77161)	1
  (18, 88826)	1
  (19, 2968)	1


In [ ]:
import hashlib
df2['hashed_directors'] = df2['directors'].apply(lambda x: int(hashlib.sha1(x.encode()).hexdigest(), 16) % (100))
df3['hashed_genres'] = df3['genres'].apply(lambda x: int(hashlib.sha1(str(x).encode()).hexdigest(), 16) % (100))


In [ ]:
display(df2['hashed_directors'])

0          44
1          85
2          85
3          85
4          44
           ..
7668276     8
7668277     8
7668278    54
7668279     4
7668280    69
Name: hashed_directors, Length: 7668281, dtype: int64

In [ ]:
df_merge = pd.merge(df3, pd.merge(df1, df2, on='tconst'), on='tconst')


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import copy
feature_columns = ['hashed_directors', 'hashed_genres']
X = df_merge[feature_columns].values
org_y = df_merge['averageRating'].values
y = copy.deepcopy(org_y)

used = (org_y > 7) | ((org_y > 3.5) & (org_y <= 6.5)) | (org_y <= 3)
y = y[used]
X = X[used]

In [ ]:
org_y = copy.deepcopy(y)

y[org_y > 7] = 2
y[(org_y > 3.5) & (org_y <= 6.5)] = 1
y[org_y <= 3] = 0
X_train, X_dev_test, y_train, y_dev_test = train_test_split(X, y, test_size=0.2)
X_dev, X_test, y_dev, y_test = train_test_split(X_dev_test, y_dev_test, test_size=0.5)


In [ ]:
print(set(y))

{0.0, 1.0, 2.0}


In [ ]:
print(X[0])

[44 79]


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
X_train = enc.fit_transform(X_train)
X_dev = enc.transform(X_dev)
X_test = enc.transform(X_test)
#X_train = scaler.fit_transform(X_train)
#X_dev = scaler.transform(X_dev)
#X_test = scaler.transform(X_test)
clf = LinearSVC(random_state=0, dual=False, verbose=1, C=128,  tol=0.000001, max_iter=10000)
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train)) 
print(clf.score(X_dev, y_dev)) 
print(clf.score(X_test, y_test)) 

[LibLinear]0.6154669314213712
0.6146865896850179
0.6121758264787551
